In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
# Path to your image
image_path = "data/15 Place Massey Saint-Bruno-de-Montarville.png"
excel_path = image_path.replace(".png", ".xlsx")

In [9]:
import base64

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Getting the base64 string
base64_image = encode_image(image_path)

In [10]:
import requests
import os
from io import StringIO

api_key = os.environ.get("OPENAI_API_KEY")

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

prompt = """
Extract the information in the table of this image.
Return the data in a format that can be ingested with Pandas. 
Just return the data, nothing else. 
Split column using ; as separator.

Code:
df = pd.read_csv(StringIO(data), header=0, names=['Pièce', 'Dimensions'])

your answer:
data =
"""

payload = {
  "model": "gpt-4-vision-preview",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": prompt
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
data = response.json()['choices'][0]['message']['content']
print(data)

data = """
Pièce;Dimensions
Hall d'entrée/Vestibule;8,10 X 4,9 p
Salon;15 X 11,5 p
Salle à manger;10,7 X 12 p
Cuisine;8,4 X 11,2 p
Chambre à coucher principale;13,6 X 10,2 p
Penderie (Walk-in);4,5 X 7,7 p
Salle de bains;13,6 X 9 p irr
Chambre à coucher;11 X 10 p
Chambre à coucher;11 X 10 p
Salle familiale;20 X 11,6 p
Bureau;11 X 8,9 p
Salle de jeux;10,10 X 17 p
Salle de bains;10,5 X 6,4 p
"""


In [11]:
import pandas as pd
df = pd.read_csv(StringIO(data.split('"""')[1]), sep=';', header=0, names=['Pièce', 'Dimensions'])
df

,Pièce,Dimensions
0,Hall d'entrée/Vestibule,"8,10 X 4,9 p"
1,Salon,"15 X 11,5 p"
2,Salle à manger,"10,7 X 12 p"
3,Cuisine,"8,4 X 11,2 p"
4,Chambre à coucher principale,"13,6 X 10,2 p"
5,Penderie (Walk-in),"4,5 X 7,7 p"
6,Salle de bains,"13,6 X 9 p irr"
7,Chambre à coucher,11 X 10 p
8,Chambre à coucher,11 X 10 p
9,Salle familiale,"20 X 11,6 p"


In [12]:
import pandas as pd
import re

def convert_comma_to_dot(text):
    # Replace comma with dot to convert to float
    return text.replace(',', '.')

def extract_numbers(text):
    # Improved regex to extract dimensions in the format 'number.number'
    numbers_dots_pattern = re.compile(r'[0-9.,]+')
    extracted = numbers_dots_pattern.findall(text)
    # Convert comma to dot
    extracted = [convert_comma_to_dot(number) for number in extracted]
    return extracted

def extract_dimensions(text):
    extracted = extract_numbers(text)
    if len(extracted) == 2:
        return extracted
    else:
        # Handle cases where the extraction does not yield exactly 2 values
        return [None, None]

# Assuming df is your DataFrame
# Apply extract_dimensions and create a new DataFrame from the results
dimensions_df = df['Dimensions'].apply(extract_dimensions).apply(pd.Series)

# Rename the new columns and add them to the original DataFrame
df['width'] = dimensions_df[0]
df['length'] = dimensions_df[1]

# convert to numeric
df['width'] = pd.to_numeric(df['width'])
df['length'] = pd.to_numeric(df['length'])

In [21]:
# Compute habitable area
habitable_area = sum(df['width'] * df['length'])
house_price = 630_000

# Compute price per square foot
price_per_square_foot = house_price / habitable_area

print(f"Price per square foot: {price_per_square_foot:.2f} $/ft²")


Price per square foot: 414.68 $/ft²


In [22]:
def save_to_excel(df, excel_path=excel_path):
    df.to_excel(excel_path, index=False)

save_to_excel(df)